# Modal Wave Climate
In this notebook, the modal wave conditions, seasonal changes and long term trends in wave climate are determined

## Dependencies

In [2]:
import numpy as np
import pandas as pd
import netCDF4
from netCDF4 import Dataset as NetCDFFile 
from netCDF4 import num2date, date2num, date2index
import datetime as dt
import re
import pymannkendall as mk

ModuleNotFoundError: No module named 'netCDF4'

## A. Import file created by Notebook1


## Arguments

- lat: latitude
- lon: longitude
- wh: significant wave height
- t: time
- qc: quality control flag, one represents "probably good data" (Ribal and Young 2019)
- back: backscatter
- ws: windspeed
- T: wave period

In [ ]:
nameCSV = 'hydro_2.csv'
df = pd.read_csv(str(nameCSV), sep=r'\s+', engine='c', header=0, na_filter=False, \
                               dtype=np.float, low_memory=False)

data = df.sort_values(by=['tt'])

lat = data.values[:,0]
lon = data.values[:,1]
wh = data.values[:,2]
tt = data.values[:,3]
qc = data.values[:,4]
back =data.values[:,5]
ws = data.values[:,6]

## B. Show Satellite Tracks

In [ ]:
fig, ax = plt.subplots()

marker_size = 10
im = ax.scatter(lon,lat, c=wh, cmap=plt.cm.jet, marker = '.')
fig.colorbar(im, ax=ax, label='Wave Height (m)')

plt.title("Satellite Tracks")
plt.xlabel("Longitude")
plt.ylabel("Latitude")            
plt.show()

## C. Find the number of days of observations

In [ ]:
days = [] #daystart
for k in range(len(tt)):
       
    t1 = netCDF4.num2date(tt[k],u'days since 1985-01-01 00:00:00 UTC')
    if k == 0: #get day 0. Then else: every day after that
        days.append(0)
        dd = netCDF4.num2date(tt[k],u'days since 1985-01-01 00:00:00 UTC')
        it = 0
    else:
        if t1.day != dd.day: #if day 1 is not equal to day 2, then append
            #print dd.day,t1.day
            days.append(k)
            it += 1
            dd = netCDF4.num2date(tt[k],u'days since 1985-01-01 00:00:00 UTC')

In [ ]:
alltime = num2date(tt[:],u'days since 1985-01-01 00:00:00 UTC')
r = netCDF4.num2date(tt[days],u'days since 1985-01-01 00:00:00 UTC')

Use days2 for graphing and making dataframes where the date, not index position, is needed

In [ ]:
days2 = [] #daystart
for k in range(len(alltime)):
       
    t1 = alltime[k]
    if k == 0: #get day 0. Then else: every day after that
        days2.append(alltime[k])
        dd =alltime[k]
        it = 0
    else:
        if t1.day != dd.day: #if day 1 is not equal to day 2, then append
            #print dd.day,t1.day
            days2.append(alltime[k])
            it += 1
            dd = alltime[k]
            
            
print 'Number of Satellite Tracks:', len(tt[days])

## D. Wave period equation 
From Govindan et al. (2011).

$ \epsilon = 3.25 \left( H_s^2 g^2/U^4 \right)^{0.31}$


In [ ]:
def waveage(H, U, grav=9.80665):
    '''
    The pseudo wave age can be expressed in terms of significant wave height and surface wind speed.
    '''

    grav2 = grav**2
    wh2 = np.square(H) 
    u4 = np.power(U,4)
    tmp = np.divide(wh2*grav2,u4)
    eps = 3.25*np.power(tmp,0.31)
    
    return eps


def waveperiod(H, U, grav=9.80665):
    '''
    Wave age from GA-2, Govindan et al.
    '''

    eps = waveage(H, U, grav=9.80665)
    period = (((eps-(5.78))/(eps+(U/(H*((U/H)+H)))))+(H+(5.70)))
    
    return period

In [ ]:
T = waveperiod(wh,ws)
print T
print len(T),len(wh)

Parameters for each day

In [ ]:
dayswh = wh[days] #wave height for each day
daysTz = T[days]
dayslon = lon[days]
dayslat = lat[days]

print len(dayswh)
print len(daysTz)
print len(dayslon)
print len(dayslat)

## E. Calculate Wave Power



### Total Wave Energy
$E = \frac{1}{8} \left(pgH_s^2\right) $


In [ ]:
def totalwaveenergy(wh):
    
    '''
    The total wave energy can be calculated using wave height, gravity and water density
    '''
    
    x = 1./8.
    p = 1025. #sea water density, kg/m^3
    g = 9.80665
    h2 = np.square(wh) 
    
    pgh = p*g*h2
    
    e = x*pgh
    
    return e

In [ ]:
we = totalwaveenergy(wh)

### Wave energy speed / Wave Velocity

# $Cg = \frac{g}{2\pi} * Tz $


In [ ]:
import math
def airywavespeed(T):
    '''
    Airy wave theory wave speed
    '''
    g = 9.80665
    pi = 2*math.pi
    gpi = g/pi
    Cg = gpi *T
    return Cg

In [ ]:
speed = airywavespeed(T)
speed

### Wave Energy Flux
$P = ECg$

In [ ]:
def waveenergyflux(H,T):
    '''
    The rate at which energy is carried by waves, determined using total wave energy and wave energy speed, kW/m
    '''
    
    e = totalwaveenergy(H)
    Cg = airywavespeed(T)
    y = 0.001 # convert from W/m to kW/m
    P = e * Cg*y
    
    return P

In [ ]:
power1 = waveenergyflux(wh,T)
pp = power1[days]

## F. Make dataframe

In [ ]:
df = pd.DataFrame(data={"date": days2, "wh":dayswh, "period":daysTz, "power":pp, "lon":dayslon, "lat":dayslat})
df.to_csv("hydro_timeseries.csv", sep=',',index=False)

### Get rolling average values based on 30 Days


In [ ]:
yrolling = df.rolling('30D', on = 'date', min_periods = 1).mean()

In [ ]:
wh_rolling=yrolling['wh']
period_rolling=yrolling['period']
power_rolling = yrolling['power']

### Incorporate rolling averages into dataframe

In [ ]:
df = pd.DataFrame(data={"date": days2, "wh":dayswh, "wh_rolling":wh_rolling, "period":daysTz, "period_rolling":period_rolling, "power":pp, "power_rolling":power_rolling })
df.to_csv("hydro_timeseries.csv", sep=',',index=False)

## G. Timeseries
### Find date when there is a data gap due to no operating satellites (will be slightly different depending on location)

In [ ]:
print df.date[57]
print df.date[58]

In [ ]:
#Define variables

#Period normal    
time1 = df.period[0:57]
time2 = df.period[58:-1]

#Period rolling
tt1 = df.period_rolling[0:57]
tt2 = df.period_rolling[58:-1]
        


#Wh normal
y1 = df.wh[0:57]
y2 = df.wh[58:-1]

#wh rolling
yy1 = df.wh_rolling[0:57]
yy2 = df.wh_rolling[58:-1]

#power normal
p1 = df.power[0:57]
p2=df.power[58:-1]

#power rolling
pp1 = df.power_rolling[0:57]
pp2 = df.power_rolling[58:-1]


#Date
x1=df.date[0:57]
x2 = df.date[58:-1]

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, ncols=1, sharex = True,figsize = (15,15))    
sns.set_style("white")

#Wh
#Before date break
ax1.plot(x1,y1,color='lightgrey',label="$H_s$")
ax1.plot(x1,yy1,color='blue',label="30-Day Average $H_s$")

#After date break
ax1.plot(x2,y2,color='lightgrey', label='False')
ax1.plot(x2,yy2,color='blue')

ax1.legend(labels=["$H_s$","30-Day Average $H_s$"], loc='upper right')
ax1.set_ylabel("H$_s$ (m)",style = 'italic',fontsize=20)
ax1.set_ylim(0,6)
ax1.tick_params(axis='y', which='both', labelsize=15)


#Wave Period 
#Before date break
ax2.plot(x1, time1, color='lightgrey')
ax2.plot(x1,tt1,color='blue')

#After date break
ax2.plot(x2,time2, color='lightgrey')
ax2.plot(x2,tt2,color='blue')

ax2.legend(labels=['T$_\mathit{z}$',"30-Day Average T$_\mathit{z}$"], loc='upper right')
ax2.set_ylabel("Tz (s)",style = 'italic',fontsize=20)
ax2.tick_params(axis='y', which='both', labelsize=15)


#Wave Power 
#Before date break
ax3.plot(x1, p1,color='lightgrey')
ax3.plot(x1,pp1,color='blue')

#After date break
ax3.plot(x2,p2,color='lightgrey')
ax3.plot(x2,pp2,color='blue')

ax3.legend(labels=['$\mathit{P}$',"30-Day Average $\mathit{P}$"], loc='upper right')
ax3.set_ylabel("P (kW/m)", style = 'italic',fontsize=20)
ax3.tick_params(axis='y', which='both', labelsize=15)
ax3.set_yticks(ticks =(0,200,400,600,800))



#Formatting
ax3.tick_params(axis='x', which='both', labelsize=15)
ax3.set_xlabel("Year", fontsize=20)
#ax3.xaxis.set_minor_locator(AutoMinorLocator())

ax3.set_xlim(min(x1),max(x2))
years = pd.date_range('2008','2015', freq='AS')

plt.savefig("Timeseries")

## H. Calculate mean, median, 95th percentile and maximum values for each parameter

In [ ]:
print 'mean wh', np.mean(df.wh)
print 'median wh', np.median(df.wh)
print '95th percentile wh', np.percentile(df.wh,95)
print 'max wh', max(df.wh)

print 'mean tz', np.mean(df.period)
print 'median tz', np.median(df.period)
print '95th percentile tz', np.percentile(df.period,95)
print 'max tz', max(df.period)

print 'mean power', np.mean(df.power)
print 'median power', np.median(df.power)
print '95th percentile power', np.percentile(df.power,95)
print 'max power', max(df.power)


## I. Find any values above a certain value
For example, find the wave height, period, power and date of each satellite record when wave height is greater than 4 m.

In [ ]:
for k in range(len(days)):
    if df.wh[k]>4:
        print df.wh[k], df.period[k], df.power[k], df.date[k]

#  Monthly values
The average wave height for each month, in each year

## I. Monthly dataframe for wh. Can use for tz, p, ws and other parameters

In [ ]:
# use this to make year/month df for each year
def getMeanH(month,year):
    valwh = []
    for k in range(len(days)):
        if r[k].year == year and r[k].month == month:
            valwh.append(wh[days[k]])

    return np.mean(valwh)

In [ ]:
my93 = []
my94 = []
my95 = []
my96 = []
my97 = []
my98 = []
my99 = []
my00 = []
my01 = []
my02 = []
my03 = []
my04 = []
my05 = []
my06 = []
my07 = []
my08 = []
my09 = []
my10 = []
my11 = []
my12 = []
my13 = []
my14 = []
my15 = []
my16 = []
my17 = []
my18 = []


for k in range(1,13):
    my93.append(getMeanH(k,1993))
    my94.append(getMeanH(k,1994))
    my95.append(getMeanH(k,1995))
    my96.append(getMeanH(k,1996))
    my97.append(getMeanH(k,1997))
    my98.append(getMeanH(k,1998))
    my99.append(getMeanH(k,1999))
    my00.append(getMeanH(k,2000))
    my01.append(getMeanH(k,2001))
    my02.append(getMeanH(k,2002))
    my03.append(getMeanH(k,2003))
    my04.append(getMeanH(k,2004))
    my05.append(getMeanH(k,2005))
    my06.append(getMeanH(k,2006))
    my07.append(getMeanH(k,2007))
    my08.append(getMeanH(k,2008))
    my09.append(getMeanH(k,2009))
    my10.append(getMeanH(k,2010))
    my11.append(getMeanH(k,2011))
    my12.append(getMeanH(k,2012))
    my13.append(getMeanH(k,2013))
    my14.append(getMeanH(k,2014))
    my15.append(getMeanH(k,2015))
    my16.append(getMeanH(k,2016))
    my17.append(getMeanH(k,2017))
    my18.append(getMeanH(k,2018))   

In [ ]:
index= ('January', 'February','March','April','May','June','July','August','September','October','November','December')
df_wh = pd.DataFrame(data={
'1993':my93,
"1994":my94,
'1993':my93,
'1994':my94,
'1995':my95,
'1996':my96,
'1997':my97,
'1998':my98,
'1999':my99,
'2000':my00,
'2001':my01,
'2002':my02,
'2003':my03,
'2004':my04,
'2005':my05,
'2006':my06,
'2007':my07,
'2008':my08,
'2009':my09,
'2010':my10,
'2011':my11,
'2012':my12,
'2013':my13,
'2014':my14,
'2015':my15,
'2016':my16,
'2017':my17,
'2018':my18}, index=index)

In [ ]:
wh_transpose =df_wh.transpose()
wh_transpose

In [ ]:
wh_stack = wh_transpose.stack()
wh_stack.to_csv("wh_years.csv", sep=',',index=1)

## Average monthly wave height plot

In [ ]:
fig, ax = plt.subplots(figsize = (15,15))
sns.boxplot(data = wh_transpose[:], palette='Spectral')
ax.set_ylabel("Significant Wave Height (m)",fontsize = 20)
ax.set_xlabel('Month',fontsize = 20)


ax.yaxis.set_tick_params(labelsize=15)
ax.xaxis.set_tick_params(labelsize=15, rotation=45)

ax.set_yticks(ticks=(0.5,1.0,1.5,2.0,2.5,3.0,3.5))

### Interannual variability Standard Deviation

In [ ]:
monthly_sd = df3.std(axis=0) # takes all january values and finds its SD. 
monthly_sd.plot()

## K. Seasonal Kendall Test
Determines the trend accounting for seasonality in data. Package from Hussain & Mahmud (2019). Slope indicates change in the parameter unit per year

In [ ]:
monthly = mk.seasonal_test(wh_stack, period=12)
monthly

## References

Govindan, R., Kumar, R., Basu, S. & Sarkar, A. (2011), ‘Altimeter-derived ocean wave period using genetic algorithm’, IEEE Geoscience and Remote Sensing Letters 8(2)

Hussain, M. M. & Mahmud, I. (2019), ‘pyMannKendall: a python package for
non parametric Mann Kendall family of trend tests’, Journal of Open Source
Software 4(39), 1556.